In [1]:
import qutip, pickle, sys
import matplotlib.pyplot as plt 
import numpy as np
import scipy.optimize as opt 
import scipy.linalg as linalg
import time as time
import math, cmath
import Max_Ent_aux_lib as me
#import proj_ev_library as projev
#import max_entev library as meev
from IPython.display import display, Math, Latex

np.set_printoptions(threshold=1.e-3,linewidth=120,precision=1, formatter={"float":lambda x: str(.001*int(1000*x)) })

Tenemos la siguiente ecuación de movimiento:

$$
    \dot{c}_M(t) I_{M}^{0}(t) = -i \bigg\{\bigg(c_{M+2}(t) - c_M(t) \bigg)\langle \rho_{M}^{0\dagger}(t) [H_{0,-2}, \rho_{M+2}^0] \rangle + \bigg(c_{M-2}(t) - c_M(t)\bigg) \langle \rho_{M}^{0\dagger}(t) [H_{0,2}, \rho_{M-2}^0] \rangle + p \langle \rho_{M}^{0\dagger}(t) [\Sigma, \rho_{M}^0 (t)]\rangle \bigg\}
$$

siendo $I_{M}^{0}(t) = \langle \rho_{M}^{0\dagger}(t) \rho_{M}^{0}(t)\rangle = I_{-M}^{0}(t)$
la cual puede reescribirse como 

$$
\dot{c}_M(t) I_{M}^{0}(t) = -i \bigg\{A_{\rho^{M, M+2}} c_{M+2}(t) - \bigg(A_{\rho^{M, M+2}} + B_{\rho^{M, M-2}} \bigg) c_{M}(t) + B_{\rho^{M, M-2}} c_{M-2}(t) \bigg\} + p C_{\rho^M} \\
= - i{\bf{\mathcal{M}}}(t) \textbf{c}(t)
$$

donde 

<ol>

<li> $A_{\rho^{M, M+2}} = \langle \rho_{M}^{0\dagger}(t) [H_{0,-2}, \rho_{M+2}^0] \rangle$ </li> 
<li> $B_{\rho^{M, M-2}} = \langle \rho_{M}^{0\dagger}(t) [H_{0,2}, \rho_{M-2}^0] \rangle$ </li>
<li> $C_{\rho^M} = \langle \rho_{M}^{0\dagger}(t) [\Sigma, \rho_{M}^0 (t)]\rangle$ </li>
    
</ol>    

Entonces, si $f(t) = \langle \rho(t) \rho^0(t) \rangle = \sum_{M} \dot{c}_M(t) I_{M}^{0}(t)$

$$
    \dot{f}(t) = \sum_{M} \bigg(\dot{c}_M(t) I_{M}^{0}(t) + {c}_M(t) \dot{I}_{M}^{0}(t) \bigg)
$$

where

1: $A=\alpha B = N(t) \exp(-m t^{1+a}) = I_M^{(0)}$ (a primer orden alpha = 1) \
2: hacer la matriz hermìtica \
3: $C = M N(t) exp(-m t^{1+a})$
    
$K(t,t') = e^{(-i M(t-t'))}$
    
con $N(t) = 1/Tr(\rho(t))$ la calculo con la definición del $\rho_M (t)=  c_M(t) \rho^0_M(t)$
con c_0(t)= 1. 

H_ {0, 2}daggger = H_{0, -2}. 
rho_{M}dagger = rho{-M}

In [65]:
def A_mmplustwo_matrix_elmt(cohrnc, time, power_law_factor = 1):
    """
    This module construct the weight of the m-th
    coherence's interaction with the (m+2)-th coherence, 
    according to the unperturbed Hamiltonian H0. 
    It takes the followings parameters as inputs:
    
    ***. the m-th coherence, labelled "cohrnc",
    ***. the time t,
    ***. and an optional real-valued power law factor for the 
         time. 
    
    It returns a real number. 
    """
    return np.e**(cohrnc * time**(-power_law_factor))

def B_mmminustwo_matrix_elmt(cohrnc, time, power_law_factor):
    """
    This module construct the weight of the m-th
    coherence's interaction with the (m-2)-th coherence, 
    according to the unperturbed Hamiltonian H0. 
    It takes the followings parameters as inputs:
    
    ***. the m-th coherence, labelled "cohrnc",
    ***. the time t,
    ***. and an optional real-valued power law factor for the 
         time. 
    
    It returns a real number. 
    """
    return np.e**(cohrnc * time**(-power_law_factor))

def C_m_matrix_elmt(cohrnc, time, power_law_factor):
    """
    This module construct the m-th coherence's self-
    interaction, associated with the perturbation Hamiltonian
    Sigma. It takes as parameters,
    
    ***. the m-th coherence, labelled "cohrnc",
    ***. the time t,
    ***. and an optional real-valued power law factor for the 
         time. 
    
    It returns a real number. 
    """
    return cohrnc * np.e**(cohrnc * time**(-power_law_factor))

def diag_mm_matrix_elmt(cohrnc, time, power_law_factor):
    """
    This module constructs the effective self-interaction for 
    the m-th coherence, due to both the unperturbed and the
    perturbation Hamiltonians. It takes as parameters:
    
    ***. the m-th coherence, labelled "cohrnc",
    ***. the time t,
    ***. and an optional real-valued power law factor for the 
         time. 
         
    It returns a real number.
    """
    return (-A_mmplustwo_matrix_elmt(cohrnc, time, power_law_factor)
            - B_mmminustwo_matrix_elmt(cohrnc, time, power_law_factor) 
            + p * C_m_matrix_elmt(cohrnc, time, power_law_factor))

def gen_func_complete_M_matrix(parameters, init_configurations, timespan, 
                                          closed_boundary_conditions = False):
    """
    This matrix constructs the explicit sparse (triangular-like)
    form of the M-matrix, wherein M_{m, m'} is the weight of 
    the m-m' coherence term. It takes the following as input:
    ***. a dictionary of parameter
         **. where, in said dictionary,
             1. the total number of coherences to be 
                considered is explicited,
             2. the strength of the Sigma-interaction 
                Hamiltonian, labelled p,
             3. and the power law factor for its submodules.
    ***. an initial configuration for the coherences at time 0,
    ***. a mesh for the times, 
    ***. a boolean option, not implemented as of yet. 
    
    ===> Returns a real-valued, triangular-like sparse matrix.
    """
    M = parameters["total_no_cohrs"]; p = parameters["p_factor"]; a = parameters["power_law_factor"]
    cm_list = init_configurations
    m_matrix_list = []; t = timespan;
    
    timet = t
    
    for m in range(M):
        if m == 0:
            m_matrix_list.append(np.array([diag_mm_matrix_elmt(m,t,a)] + [0]
                                    + [A_mmplustwo_matrix_elmt(m+2,t,a)] + [0 for k in range(M-3)])) 
        if m == 1:
            local_list = [0] + [diag_mm_matrix_elmt(m,t,a)] + [0] + [A_mmplustwo_matrix_elmt(m+2, t, a)]
            local_length = len(np.array(local_list))
            m_matrix_list.append(local_list + [0 for k in range(M-local_length)])
        if (m > 1) and (m < M - 2):  
            local_list = [0 for j in range(m-2)]; local_length = len(local_list)
            local_list += ([B_mmminustwo_matrix_elmt(-m-2,t,a)] + [0] + [diag_mm_matrix_elmt(m,t,a)] + [0] 
                        + [A_mmplustwo_matrix_elmt(m+2,t,a)])
            local_list += [0 for j in range(M-local_length - 5)]
            m_matrix_list.append(np.array(local_list))
        if m == M - 2: 
            m_matrix_list.append(np.array([0 for i in range(m-2)] + [B_mmminustwo_matrix_elmt(-m-2,t,a)] + [0] 
                                 + [diag_mm_matrix_elmt(m,t,a)] + [0]))
        if m == M - 1:
            m_matrix_list.append(np.array([0 for i in range(m-2)] + [B_mmminustwo_matrix_elmt(-m-2,t,a)] + [0] 
                                 + [diag_mm_matrix_elmt(m,t,a)]))
            
    return qutip.Qobj(m_matrix_list)

def gen_func_even_cohr_M_matrix(parameters, init_configurations, timet, 
                                          closed_boundary_conditions = False,
                                          visualization = False,
                                          as_qutip_qobj = False):
    """
    This module constructs the even-coherences weight matrix,
    M, wherein M_{m, m'} is the weight of the m-m' coherence 
    term, with both m and m' even numbered-coherences. The odd
    coherences are disregardes for these do not factor into the 
    M-matrix. It takes the following as input:
    ***. a dictionary of parameters
         **. where, in said dictionary,
             1. the total number of even coherences to be 
                considered is explicited,
             2. the strength of the Sigma-interaction 
                Hamiltonian, labelled p,
             3. and the power law factor for its submodules.
    ***. an initial configuration for the coherences at time 0,
    ***. a mesh for the times, 
    ***. a boolean option, not implemented as of yet. 
    ***. a boolean option for visualizing a
             1. plot of the M-matrix's eigenvalues,
             2. and the M-matrix coefficients in a Hinton
                diagram.
    ***. a boolean option for returning the M-matrix as 
         quantum object.     
    
        ===> Returns a real-valued matrix. 
    *Note that his module does not return a sparse matrix. 
    """
    M = parameters["total_no_cohrs"]; p = parameters["p_factor"]; a = parameters["power_law_factor"]
    cm_list = init_configurations
    m_matrix_list = []; t = timet;
    
    for m in range(M):
        if m == 0:
            local_array = np.array([A_mmplustwo_matrix_elmt(cohrnc = m, time = t, 
                                                                   power_law_factor = a) + 
                                           p * C_m_matrix_elmt(cohrnc = m, time = t, 
                                                                   power_law_factor = a)] 
                                          + [A_mmplustwo_matrix_elmt(cohrnc = m+2, time = t, 
                                                                   power_law_factor = a)] 
                                          + [0 for i in range(M-2)])
            local_array = local_array/(sum(local_array))
            m_matrix_list.append(local_array)
            local_array = None
        if (m > 0 and m < M-1):
            list_with_zeros = [0 for i in range(m-1)]
            local_array = np.array(list_with_zeros
                                          + [B_mmminustwo_matrix_elmt(cohrnc = m, time = t, power_law_factor = a)]
                                          + [diag_mm_matrix_elmt(cohrnc = m, time = t, power_law_factor = a)] 
                                          + [A_mmplustwo_matrix_elmt(cohrnc = m+2, time = t, power_law_factor = a)]  
                                          + [0 for i in range(M - (len(list_with_zeros)+3))])
            local_array = local_array/(sum(local_array))
            m_matrix_list.append(local_array)
            local_array = None       
        if (m == M-1):
            local_array = np.array([0 for i in range(M - 2)] 
                                          + [B_mmminustwo_matrix_elmt(cohrnc = m, time = t, power_law_factor = a)] 
                                          + [B_mmminustwo_matrix_elmt(cohrnc = m, time = t, power_law_factor = a) + 
                                               p * C_m_matrix_elmt(cohrnc = m, time = t, power_law_factor = a)])
            local_array = local_array/(sum(local_array))
            m_matrix_list.append(local_array)
            local_array = None
    
    ### test:
    dimensions_equal_tot_no_cohr = [len(m_matrix_list) == M for i in range(len(m_matrix_list))]
    assert (np.all(dimensions_equal_tot_no_cohr and len(m_matrix_list) == M)),"Error: M-matrix is not square"
        
    if visualization:
        eigenvalues_list = linalg.eig(m_matrix_list)[0]
        plt.scatter([i+1 for i in range(len(eigenvalues_list))], np.array(eigenvalues_list), label = "M-matrix's eigenvalues")
        plt.matshow(m_matrix_list)
        
    if as_qutip_qobj:
        try: 
            m_matrix_list = qutip.Qobj(m_matrix_list)
        except NameError:
            print(NameError)
        
    return m_matrix_list

def generating_function_complete_M_matrix(parameters, init_configurations, timespan, 
                                          even_cohr_matrix_only = True):
    """
    This module construct the triangular-like matrix M,
    whose exponential yields the solution to the Markovian
    system of differential equations. It takes as parameters:
    
    ***. a dictionary of parameters, containing the total number 
           of coherences, the p-factor due to the unperturbed 
           Hamiltonian H_0, and the power-law factor.
    ***. An initial coherences' configuration,
    ***. A list of desired times 
    ***. A boolean option, not-implemented as of yet.
    """
    
    if even_cohr_matrix_only:
        try:
            m_matrix = gen_func_even_cohr_M_matrix(parameters, init_configurations, timespan, 
                                          closed_boundary_conditions = False,
                                          visualization = False,
                                          as_qutip_qobj = False)
        except NameError:
            print(NameError)
    else:
        try: 
            m_matrix = gen_func_complete_M_matrix(parameters, init_configurations, timespan, 
                                          closed_boundary_conditions = False)
        except NameError:
            print(NameError)
    
    return m_matrix

In [66]:
p = .5; a = 1; M = 10; coherences_t0_pert0 = 1.
param_list = {"total_no_cohrs": M, "p_factor": p, "power_law_factor": a}
coeff_list_t0 = np.array([coherences_t0_pert0 - np.random.rand() for i in range(param_list["total_no_cohrs"])])
M_tensor_f= np.array(gen_func_even_cohr_M_matrix(param_list, coeff_list_t0, 1, False, False, False))

In [67]:
M_tensor_f.dot(coeff_list_t0)

array([0.851, 0.23, 0.126, ..., 0.506, 0.554, 0.562])

In [62]:
from scipy.integrate import odeint

def differential_system(cohr_vector, t, M_tensor):
    return M_tensor.dot(cohr_vector)

ts = np.linspace(0., 10., 100)
result = odeint(func = differential_system, 
                y0 = coeff_list_t0, 
                t = ts,
                args = (M_tensor_f, p))

TypeError: differential_system() takes 3 positional arguments but 4 were given

In [29]:
qutip.Qobj(M_tensor_f)
#assert me.M_matrix_test(M_tensor_f), "ERROR: M-matrix not next-diagonal triangular."

Quantum object: dims = [[10], [10]], shape = (10, 10), type = oper, isherm = False
Qobj data =
[[0.11900000000000001 0.88 0.0 ... 0.0 0.0 0.0]
 [0.145 -0.217 1.072 ... 0.0 0.0 0.0]
 [0.0 0.135 -0.135 ... 0.0 0.0 0.0]
 ...
 [0.0 0.0 0.0 ... 0.151 0.747 0.0]
 [0.0 0.0 0.0 ... 0.096 0.192 0.711]
 [0.0 0.0 0.0 ... 0.0 0.153 0.846]]

In [ ]:
Preguntas:
    
    1. La matrix M tiene que depender con el tiempo me parece, si A, B y C lo hacen
    2. Como tendría que implementar la norma???
    3. Empezar a jugar con los parámetros. 
    
m+2 vs m+2
** reescribir la matriz en tèrminos de los c_pares.

In [45]:
def generating_function_even_cohr_M_matrix(parameters, init_configuration, timespan, 
                                           closed_boundary_conditions = False):
    
    M = parameters["total_no_cohrs"]; p = parameters["p_factor"]; a = parameters["power_law_factor"]
    cm_list = init_configuration
    m_matrix_list = []; t = timespan;
    
    for m in range(M):
        if m == 0:
            m_matrix_list.append(np.array([diag_mm_matrix_elmt(m,t,a)] 
                                    + [A_mmplustwo_matrix_elmt(m+2,t,a)] + [0 for k in range(int((M-3)/2))])) 
        if m == 1:
            local_list = [0] + [diag_mm_matrix_elmt(m,t,a)] + [A_mmplustwo_matrix_elmt(m+2, t, a)]
            local_length = len(np.array(local_list))
            m_matrix_list.append(np.array(local_list + [0 for k in range(int(M/2 + 1)-local_length)]))
        if (m > 1) and (m < M - 2):  
            local_list = [0 for j in range(m-2)]; local_length = len(local_list)
            local_list += ([B_mmminustwo_matrix_elmt(-m-2,t,a)] + [diag_mm_matrix_elmt(m,t,a)] 
                        + [A_mmplustwo_matrix_elmt(m+2,t,a)])
            local_list += [0 for j in range(int(M/2+1)-local_length - 5)]
            m_matrix_list.append(np.array(local_list))
        if m == M - 2: 
            m_matrix_list.append(np.array([0 for i in range(int(M/2)-2)] + [B_mmminustwo_matrix_elmt(-m-2,t,a)]  
                                 + [diag_mm_matrix_elmt(m,t,a)] + [0]))
        if m == M - 1:
            m_matrix_list.append(np.array([0 for i in range(int(M/2)-2)] + [B_mmminustwo_matrix_elmt(-m-2,t,a)] + [0] 
                                 + [diag_mm_matrix_elmt(m,t,a)]))
    return (m_matrix_list)


In [4]:
A = np.array([-1,2,3])
sum(A)

4

In [ ]:
# Si la matrix M(t) = M para todo t, uso la propiedad de semigrupo y ya fue

def semigroup_phit_and_rhot_sol(coeff_list_t0, Mtensor, ts):
    coeff_at_timet_list = []
    coeff_at_timet_list.append(np.array(coeff_list_t0))
    
    for i in range(1, len(ts)-1):
        try: 
            a = (ts[i+1] * Mtensor).expm() * coeff_list_t0[0]
            coeff_at_timet_list.append(a)
        #if (Exception):
        #   continue 
    return coeff_at_timet_list    

# Si la matrix M(t) fuese no constante, odeints

from scipy.integrate import ode
import random

def F(y, t, Mtensor):
    return Mtensor.dot(y)

real_Mtensor = (np.real(Mtensor))
ts1 = np.linspace(0, 500, 200)
result = odeint(func=F, y0=coeff_list_t0, t= ts1 , args=(real_Mtensor,))
plt.figure(figsize=(7, 4))

for i in range(len(total_no_spins)):
    plt.plot(ts1, result[:,i], label=r'$c_{(%d)}(t)$'%i)

plt.xlabel("t")
plt.ylabel("Phi(t)")
plt.legend()

